<a href="https://colab.research.google.com/github/cwinsor/medical_image_uw_madison/blob/main/colab_01_data_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing
This notebook runs data_preprocess.py

In [15]:
!pwd

/content


In [28]:
import sys
import os
from shutil import rmtree

In [35]:
# !rm -rf medical_image_uw_madison
# os.removedirs('medical_image_uw_madison')
# if os.path.exists for
if os.path.isdir('/content/medical_image_uw_madison'):
    print('removing...')
    rmtree('/content/medical_image_uw_madison')
else:
    print('not there')


not there


In [ ]:

if 'google.colab' in sys.modules:
    print("host is colab")
    !git clone --quiet https://github.com/cwinsor/medical_image_uw_madison.git
    os.chdir('/content/medical_image_uw_madison/')
    print(os.getcwd())
    print(os.listdir())
    print(os.scandir())
    # !ls -la
    !git submodule update --init
    # !cd /content
    os.chdir('/content/')
    sys.path.append('/content/medical_image_uw_madison/  ')
else:
    print("host is traditional server")
    assert False, "not coded"

host is colab
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: could not create work tree dir 'medical_image_uw_madison': No such file or directory


In [5]:
import os
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import glob
from tqdm.auto import tqdm
import albumentations as A

In [10]:
# torch not needed for preprocessing...
# import torch
# print(torch.__version__);print(torch.version.cuda)

In [6]:
def rle_decode(mask_rle, shape):
    s = np.array(mask_rle.split(), dtype=int)
    starts, lengths = s[0::2] - 1, s[1::2]
    ends = starts + lengths
    h, w = shape
    img = np.zeros((h * w,), dtype = np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img.reshape(shape)

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
os.chdir("./data/tract")

df_train = pd.read_csv("./train.csv")
df_train = df_train.sort_values(["id", "class"]).reset_index(drop = True)
df_train["patient"] = df_train.id.apply(lambda x: x.split("_")[0])
df_train["days"] = df_train.id.apply(lambda x: "_".join(x.split("_")[:2]))
num_slices = len(np.unique(df_train.id))
num_empty_slices = df_train.groupby("id").apply(lambda x: x.segmentation.isna().all()).sum()
num_patients = len(np.unique(df_train.patient))
num_days = len(np.unique(df_train.days))
print({
    "#slices:": num_slices,
    "#empty slices:": num_empty_slices,
    "#patients": num_patients,
    "#days": num_days
})